In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gc
from pathlib import Path
import numpy as np
import yaml

import torch
from torchsummary import summary
from pytorch_lightning.trainer import seed_everything

from turboflow.models.phyrff import plDivFreeRFFNet
from turboflow.dataloaders import TurboFlowDataModule
from turboflow.utils import phy_utils as phy
from turboflow.utils import torch_utils as tch

import matplotlib.pyplot as plt

In [3]:
base_dir = Path('/','home','dicarlo_d','Documents','Code','TurboSuperResultion','recipes','time_cnn')
data_dir = Path('/','home','dicarlo_d','Documents','Datasets','Turb2D.hdf5')
fig_path = base_dir / Path('figures')
res_path = base_dir / Path('results')

In [8]:
results_file = res_path.glob('*.yaml')

print('Which file?')
results_file = sorted(list(results_file))
for f, file in enumerate(results_file):
    name = file.name
    print(f'[{f}]\t{name}')

results_yml = results_file[6]
print(results_yml)
seed = 666

Which file?
[0]	results_32x32_mymult.yaml
[1]	results_32x32_sifan.yaml
[2]	results_CNN_32x32.yaml
[3]	results_CNN_32x32_1conv.yaml
[4]	results_CNN_32x32_2conv.yaml
[5]	results_CNN_32x32_4conv.yaml
[6]	results_SGU_1x32x32_TcatS.yaml
[7]	results_SGU_32x32_4conv.yaml
[8]	results_SGU_32x32_TcatS.yaml
[9]	results_SGU_32x32_TxS.yaml
[10]	results_SGU_32x32_small.yaml
/home/dicarlo_d/Documents/Code/TurboSuperResultion/recipes/time_cnn/results/results_SGU_1x32x32_TcatS.yaml


In [9]:
with open(results_yml, 'r') as file:
    result = yaml.safe_load(file)

best_model_path = result['best_model_path']
exp_name = result['exp_name']

seed_everything(seed, workers=True)
model = plDivFreeRFFNet.load_from_checkpoint(best_model_path)
model.cuda()

summary(model, (1, 3), batch_size=16, device='cuda')

Global seed set to 666


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Fourier-1                  [16, 256]               0
           Fourier-2                  [16, 256]               0
         LayerNorm-3             [16, 256, 256]             512
            Linear-4             [16, 256, 256]          65,792
         LayerNorm-5             [16, 256, 128]             256
            Conv1d-6             [16, 256, 128]          65,792
 SpatialGatingUnit-7             [16, 256, 128]               0
            Linear-8             [16, 256, 256]          33,024
         gMLPBlock-9             [16, 256, 256]               0
        LayerNorm-10             [16, 256, 256]             512
           Linear-11             [16, 256, 256]          65,792
        LayerNorm-12             [16, 256, 128]             256
           Conv1d-13             [16, 256, 128]          65,792
SpatialGatingUnit-14             [16, 2

In [ ]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

model.cnn.conv1.register_forward_hook(get_activation('conv1'))
model.rff_space.register_forward_hook(get_activation('rff_space'))
model.rff_time.register_forward_hook(get_activation('rff_time'))

print(activation)

In [10]:
batch_size = {
    'train' : 128
,   'val'   : 128
}
dx = {
    'train' : 8
,   'val'   : 4
}
dt = {
    'train' : 8 
,   'val'   : 4
}
nt_train = 32


# load dataset for TRAIN
dm_train = TurboFlowDataModule(
    dataset='Turb2D', 
    data_dir=data_dir,
    time_idx=np.arange(dt['train']*nt_train),

    train_batch_size=128,
    val_batch_size=128,
    test_batch_size=128,

    train_downsampling_space=dx['train'],
    val_downsampling_space=dx['val'],
    test_downsampling_space=64,

    train_downsampling_time=dt['train'],
    val_downsampling_time=dt['val'],
    test_downsampling_time=16,

    train_shuffle=True,
    val_shuffle=False,
    test_shuffle=False,
    num_workers=8)

dm_train.setup(stage='fit')

In [11]:
datasets = [dm_train.train_dataset, dm_train.val_dataset]

for dataset in datasets:
    X, y = dataset[:]
    print(X.shape, y.shape, dataset.img_res, dataset.vars_shape_img)

torch.Size([32768, 3]) torch.Size([32768, 2]) 32 (32, 32, 32, 3)
torch.Size([262144, 3]) torch.Size([262144, 2]) 64 (64, 64, 64, 3)


In [12]:
X, y = datasets[0][:]

print(X.shape, y.shape)
print(X)

y_hat = model(X.to(model.device))

torch.Size([32768, 3]) torch.Size([32768, 2])
tensor([[0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0314],
        [0.0000, 0.0000, 0.0627],
        ...,
        [0.9725, 0.9725, 0.9098],
        [0.9725, 0.9725, 0.9412],
        [0.9725, 0.9725, 0.9725]])


RuntimeError: CUDA out of memory. Tried to allocate 8.00 GiB (GPU 0; 15.75 GiB total capacity; 8.10 GiB already allocated; 6.50 GiB free; 8.10 GiB reserved in total by PyTorch)

In [ ]:
X, y = datasets[0][32000:32004]

print(X.shape, y.shape)
print(X)

output = model(X.to(model.device))

In [ ]:
out_conv1 = activation['conv1']
out_rff_space = activation['rff_space']
out_rff_time = activation['rff_time']
print(out_conv1.shape)
print(out_rff_space.shape)
print(out_rff_time.shape)

In [ ]:
B, C, H, W = out_conv1.shape

B = 4
C = 4
fig, axarr = plt.subplots(1, 4, figsize=(20,5))
for b in range(4):
    x = out_rff_space[b,None,:]
    t = out_rff_time[b,:,None]
    axarr[b].imshow((t*x).cpu())
plt.tight_layout()
plt.show()



fig, axarr = plt.subplots(B,C, figsize=(10,10))
x = out_conv1
x = torch.max_pool2d(x, (2,2))
for b in range(B):
    for c in range(C):
        axarr[b,c].imshow(x[b,c,:,:].cpu())

plt.tight_layout()
plt.show()